<a href="https://colab.research.google.com/github/yunjeooong/MiraeassetFin/blob/main/1%E1%84%8E%E1%85%A1_%E1%84%8B%E1%85%AD%E1%84%8B%E1%85%A3%E1%86%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import my_settings
import pandas as pd
import os

class ClovaSummary:
    # Clova Speech invoke URL
    url = 'https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize'
    client_id = my_settings.CLIENT_ID
    client_secret = my_settings.CLIENT_SECRET

    def req(self, content):
        request_body = {
            "document": {
                "content": content
            },
            "option": {
                "language": 'ko',
                "model": "general",
                "summaryCount": 2,
                "tone": 3
            }
        }
        headers = {
            'Accept': 'application/json;UTF-8',
            'Content-Type': 'application/json;UTF-8',
            'X-NCP-APIGW-API-KEY-ID': self.client_id,
            'X-NCP-APIGW-API-KEY': self.client_secret
        }
        return requests.post(headers=headers,
                             url=self.url,
                             data=json.dumps(request_body).encode('UTF-8'))

def split_text(text, max_length):
    words = text.split()
    chunks = []
    chunk = ""
    for word in words:
        if len(chunk) + len(word) + 1 > max_length:
            chunks.append(chunk)
            chunk = word
        else:
            if chunk:
                chunk += " "
            chunk += word
    chunks.append(chunk)
    return chunks

def clean_summary(summary):
    return summary.replace('@', '').replace('\n', ' ').strip()

if __name__ == '__main__':
    # 엑셀 파일 절대 경로
    excel_file_path = r"/content/filtered_news_생활_문화.xlsx"

    # 엑셀 파일 읽기
    df = pd.read_excel(excel_file_path)

    # 데이터프레임의 열 이름 확인
    print("Column names:", df.columns)

    max_length = 2000  # Naver Summary API의 텍스트 길이 제한을 고려한 최대 길이

    # Initialize a list to store the summaries
    summaries = []

    # Verify the column name
    if '본문' not in df.columns:
        raise KeyError("Column '본문' does not exist in the Excel file. Available columns are: " + ", ".join(df.columns))

    for index, row in df.iterrows():
        content = row['본문']

        # 텍스트를 분할
        content_chunks = split_text(content, max_length)

        summary = ""
        for chunk in content_chunks:
            if len(chunk.strip()) < 50:  # 짧은 텍스트는 건너뛰기
                continue
            res = ClovaSummary().req(chunk)
            rescode = res.status_code
            if rescode == 200:
                summary_chunk = json.loads(res.text)["summary"]
                summary += clean_summary(summary_chunk) + " "
            else:
                print(f"Error for Article {index+1} : " + res.text)

        # 최종 요약
        if len(content_chunks) > 1:
            res = ClovaSummary().req(summary)
            rescode = res.status_code
            if rescode == 200:
                summary = json.loads(res.text)["summary"]
                summary = clean_summary(summary)
            else:
                print(f"Error for Article {index+1} : " + res.text)

        summaries.append(f"{index+1}번 기사 요약: {summary.strip()}")
        print(f"Article {index+1} 요약:\n{summary.strip()}\n")

    # Add the summaries to the dataframe
    df['요약'] = summaries

    # Save the summarized results to a new Excel file
    output_dir = r"/content/"
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, "naver_news_content_summarized_0722_생활문화.xlsx") #파일명은 예상 분류명
    df.to_excel(output_file, index=False)

    print(f"요약된 결과가 {output_file}에 저장되었습니다.")


Column names: Index(['뉴스제목', '본문', 'predicted_section'], dtype='object')
Error for Article 1 : {"status":400,"error":{"errorCode":"E100","message":"Insufficient valid sentence"}}
Article 1 요약:
밸류업 대신 밸류킬 기업가치보다 훨씬 낮게 공개매수가 정해 잇단 피해 주주환원 인색했던 기업 상장폐지 후 고배당 '눈총' 두산밥캣 합병 외국인도 비판 금융위원장 후보자 "개선 검토" 일본 증시 시세판 앞 지나가는 남성 [사진 = AP 연합뉴스. 올해 사모펀드들이 공개매수 후 상장폐지를 추진하고 있는 건에 대해선 소액주주들은 배당도 제대로 주지 않고 낮은 주가를 방치하고 있다가 상폐를 추진하다고 불만이 높음. 시총은 7000억원, 밥캣은 15조원이므로 둘 사이의 합병은 4:96 비율로 이뤄졌어야 한다"고 말함. 이날 국회 인사청문회에 참석한 김병환 금융위원장 후보자는 두산그룹사업 구조 개편 논란에 대해 "시장의 우려가 있는 것을 알고 있음.

Article 2 요약:
BC카드, 국내거주 외국인근로자 데이터 분석 '코리안드림' 대신 '마이드림' 이제는 '코리안 드림(Korean Dream)'이 아닌 '마이 드림(My Dream)' 국내에 거주하고 있는 외국인 근로자들이 부모나 형제, 배우자 등 가족 송금보다는 본인 계좌에 송금하는 사례가 크게 증가한 것으로 나타남. 22일 BC카드가 해외송금 전문 핀테크사인 글로벌머니익스프레스(GME)와 협업해 국내 거주 외국인 100만여명의 송금과 소비데이터를 분석한 결과, 지난 2021년 부모와 형제, 배우자에게 송금하는 비율은 과반 이상인 58%를 차지했지만, 2023년 코로나 팬데믹 이후 처음으로 본인 송금 비율이 20%를 돌파, 올해는 25%로 2021년 대비 10%p 상승한 것으로 나타남.

Article 3 요약:
두산밥캣 로고. /두산밥캣 제공 두산밥캣이 기존 자사주에 더해 두